# Neutron Powder Diffraction

In this tutorial demonstrates how neutron-scattering data can be loaded, visualized, and manipulated with generic functionality from `scipp` as well as neutron-specific functionality from `scipp.neutron`. It focuses on reducing data from the ORNL [POWGEN](https://neutrons.ornl.gov/powgen) neutron diffractometer.

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

### Loading Nexus files

Loading Nexus files requires Mantid.
See, e.g., [Installation](https://scipp.github.io/getting-started/installation.html) on how to install scipp and Mantid with `conda`.
We are using two files in this tutorial,
[PG3_4844_event.nxs](http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e)
and
[PG3_4866_event.nxs](http://198.74.56.37/ftp/external-data/MD5/3d543bc6a646e622b3f4542bc3435e7e).
Both are available as part of Mantid's test data.

We start by loading two files: the sample and the vanadium runs.

In [ ]:
sample = sc.neutron.load(filename='PG3_4844_event.nxs',
                         load_pulse_times=False,
                         mantid_args={'LoadMonitors': True})
vanadium = sc.neutron.load(filename='PG3_4866_event.nxs',
                           load_pulse_times=False)

The optional `mantid_args` dict is forwarded to the Mantid algorithm used for loading the files &ndash; in this case [LoadEventNexus](https://docs.mantidproject.org/nightly/algorithms/LoadEventNexus-v1.html) &ndash; and can be used to control, e.g., which part of a file to load.
Here we request loading monitors, which Mantid does not load by default.
The resulting dataset looks as follows:

In [ ]:
sample

In [ ]:
sc.show(sample)

### Instrument view

Scipp provides a rudimentary version of the Mantid [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View), which can be used to take a quick look at the neutron counts on the detector panels in 3D space or using various cylindrical and spherical projections

In [ ]:
sc.neutron.instrument_view(sample)

### Plot against scattering angle $\theta$ using `groupby`

*This is not an essential step and can be skipped.*

Plotting raw data directly yields a hard-to-interpret figure.
We can obtain something more useful by "binning" the spectrum axis based on its $\theta$ value, using the split-apply-combine approach provided by `groupby`:

In [ ]:
sample.coords['theta'] = sc.neutron.scattering_angle(sample)
vanadium.coords['theta'] = sc.neutron.scattering_angle(vanadium)
theta_bins = sc.Variable(['theta'],
                         unit=sc.units.rad,
                         values=np.linspace(0.0, np.pi/2, num=2000))

In [ ]:
theta_sample = sc.groupby(
    sample,
    'theta',
    bins=theta_bins).flatten('spectrum')

<div class="alert alert-info">

**Note**
    
Use `sum` instead of `flatten` when working with dense (histogrammed) data
</div>

In [ ]:
theta_sample

In [ ]:
tof_bins = sc.Variable(dims=['tof'], unit=sc.units.us, values=np.linspace(0.0, 17000.0, num=1000))
plot(theta_sample, bins={'tof':tof_bins})

### Unit conversion

*Note: We are back to working with `sample`, not `theta_sample`.*

`scipp.neutron` provides means to convert between units (dimensions) related to time-of-flight.
The loaded raw data has `Dim.Tof`, and we convert to interplanar lattice spacing (d-spacing):

In [ ]:
dspacing_vanadium = sc.neutron.convert(vanadium, 'tof', 'd-spacing')
dspacing_sample = sc.neutron.convert(sample, 'tof', 'd-spacing')
dspacing_sample

### Neutron monitors

*Processing after this section does not continue based on the monitor-normalized data produced here.
This section could thus be skipped.*

If available, neutron monitors are stored as attributes of a data array:

In [ ]:
mon = sample.attrs['monitor1'].value
mon

The monitor could, e.g., be used to normalize the data.
To do so, both data and monitor need to be converted to a unit other than time-of-flight, e.g., wavelength or energy:

In [ ]:
sample_lambda = sc.neutron.convert(sample, 'tof', 'wavelength')
mon = sc.neutron.convert(mon, 'tof', 'wavelength')

The sample data is in event-mode, i.e., is not histogrammed.
Event data *can* be divided by a histogram (such as `mon` in this case), but it needs to be explicitly aligned to the correct coordinates first (see [Unaligned data](../user-guide/unaligned-data.rst)).
This alignment does *not* actually perform the conversion to a histogram:

In [ ]:
edges = sc.Variable(dims=['wavelength'], unit=sc.units.angstrom, values=np.linspace(0, 1, num=1000))
sample_lambda.realign({'wavelength':edges})

We also rebin the monitor since the original binning is very fine:

In [ ]:
mon = sc.rebin(mon, 'wavelength', edges)
mon

The sample data is now *realigned event data* and the monitor is a histogram.
Multiplication and division operations for such cases are supported by modifying the weights (values) for each event:

In [ ]:
sample_over_mon = sample_lambda / mon
sample_over_mon

Note that no data values are displayed since the internal representation is still based on unaligned event data.
The underlying data can be accessed via the `unaligned` property:

In [ ]:
sample_over_mon.unaligned

Finally, we can plot the event data with on-the-fly binning.
In this case specifying bin edges for the event data is not necessary since the realigned wrapper contains the required bin-edge coordinate:

In [ ]:
plot(sample_over_mon)

### From events to histogram

*Note: We are continuing here with data that has not been normalized to the monitors.*

We histogram the event data:

In [ ]:
dspacing_bins = sc.Variable(
    ['d-spacing'],
    values=np.arange(0.3, 2.0, 0.001),
    unit=sc.units.angstrom)
hist = sc.Dataset({'sample':sc.histogram(dspacing_sample, dspacing_bins),
                   'vanadium':sc.histogram(dspacing_vanadium, dspacing_bins)})
sc.show(hist['spectrum', 0:3]['d-spacing', 0:7])

In [ ]:
plot(hist)

### Summing (focussing) and normalizing

After conversion to `Dim.DSpacing`, generic `sum` and `/` operations can be used to "focus" and normalize the diffraction data to the vanadium run:

In [ ]:
summed = sc.sum(hist, 'spectrum')
plot(summed)

In [ ]:
normalized = summed['sample'] / summed['vanadium']
plot(normalized)

### Focussing with $\theta$ dependence in event-mode

Instead of focussing all data into a single spectrum, we can use `groupby` to focus each of multiple groups of spectra into a distinct output spectrum.
Here we define groups based on a range of scattering angles &ndash; a simple $\theta$-dependent binning.
This also demonstrates how we can postpone histogramming until after the focussing step.

In [ ]:
theta = sc.Variable(['scattering_angle'],
                    unit=sc.units.rad,
                    values=np.linspace(0.0, np.pi/2, num=16))

# Option 1: Realign, then group dataset, end result event data
dspacing = sc.Dataset({
    'sample':sc.realign(dspacing_sample, {'d-spacing':dspacing_bins}),
    'vanadium':sc.realign(dspacing_vanadium, {'d-spacing':dspacing_bins})
})
focussed = sc.groupby(dspacing, 'theta', bins=theta).sum('spectrum')
normalized = focussed['sample'] / sc.histogram(focussed['vanadium'])

# # Option 2: Focus and histogram independently, then insert into dataset, end result histogram
# focussed_sample = sc.groupby(
#    dspacing_sample,
#    'theta',
#    bins=theta).flatten('spectrum')
# focussed_vanadium = sc.groupby(
#    dspacing_vanadium,
#    'theta',
#    bins=theta).flatten('spectrum')
# focussed = sc.Dataset({'sample':sc.histogram(focussed_sample, dspacing_bins),
#                   'vanadium':sc.histogram(focussed_vanadium, dspacing_bins)})
# normalized = focussed['sample'] / focussed['vanadium']

The normalization output is still in event-mode, so data is histogrammed for plotting:

In [ ]:
plot(normalized)

As a bonus, we can use slicing and a dict-comprehension to quickly create of plot comparing the spectra for different scattering angle bins:

In [ ]:
# compute centers of theta bins
angles = normalized.coords['scattering_angle'].values
angles = 0.5*(angles[1:] + angles[:-1])
plot(sc.Dataset(
    {
        '{}'.format(angles[group]):
        normalized['d-spacing', 300:500]['scattering_angle', group]
        for group in range(2,6)
    }))